In [1]:
import pandas as pd
import numpy as np

In [2]:
ecdc = pd.read_csv('data/ecdc_covid19_20200418.csv', parse_dates=['dateRep'], dayfirst=True)
interest = pd.read_csv('data/interest_over_time.csv', parse_dates=['date'])
trends_title =  pd.read_csv('data/gtrends_dict_trend.csv', index_col=0, parse_dates=['date'])
trends_traffic =  pd.read_csv('data/gtrends_dict_traffic.csv', index_col=0, parse_dates=['date'])
related_queries = pd.read_csv('data/related_queries.csv', index_col=0, parse_dates=['date'])

In [3]:
ecdc.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018
0,2020-04-18,18,4,2020,51,1,Afghanistan,AF,AFG,37172386.0
1,2020-04-17,17,4,2020,10,4,Afghanistan,AF,AFG,37172386.0
2,2020-04-16,16,4,2020,70,2,Afghanistan,AF,AFG,37172386.0
3,2020-04-15,15,4,2020,49,2,Afghanistan,AF,AFG,37172386.0
4,2020-04-14,14,4,2020,58,3,Afghanistan,AF,AFG,37172386.0


In [4]:
interest.head()

,date,coronavirus,Country
0,2019-12-31,0,Australia
1,2020-01-01,0,Australia
2,2020-01-02,0,Australia
3,2020-01-03,0,Australia
4,2020-01-04,0,Australia


In [5]:
trends_traffic.head()

,date,country_code,0
0,2020-03-26,AU,"{0: '20K+', 1: '20K+', 2: '10K+', 3: '10K+', 4..."
1,2020-03-26,GB,"{0: '500K+', 1: '500K+', 2: '500K+', 3: '200K+..."
2,2020-03-26,IE,"{0: '20K+', 1: '20K+', 2: '10K+', 3: '5K+', 4:..."
3,2020-03-26,NZ,"{0: '20K+', 1: '10K+', 2: '5K+', 3: '5K+', 4: ..."
4,2020-03-26,US,"{0: '1M+', 1: '200K+', 2: '200K+', 3: '200K+',..."


In [6]:
trends_title.head()

,date,country_code,0
0,2020-03-26,AU,"{0: 'Powerball tonight', 1: 'Powerball winner'..."
1,2020-03-26,GB,"{0: 'Chester Zoo', 1: 'Until Tomorrow', 2: 'Ri..."
2,2020-03-26,IE,"{0: 'Bugzy Malone', 1: 'Mark Blum', 2: 'Pentat..."
3,2020-03-26,NZ,"{0: 'Prince Charles', 1: 'Until Tomorrow', 2: ..."
4,2020-03-26,US,"{0: 'Mark Blum', 1: 'Carole Baskin', 2: 'Lion'..."


In [7]:
related_queries.head()

,date,coronavirus,Country,related_queries
82,2020-03-22,100,Australia,"{0: 'nsw lockdown coronavirus', 1: 'access sup..."
182,2020-03-12,100,Canada,"{0: 'tom hanks coronavirus', 1: 'sophie trudea..."
183,2020-03-13,99,Canada,"{0: 'lcbo closing coronavirus', 1: 'trudeau wi..."
292,2020-03-12,100,Ireland,"{0: 'tom hanks coronavirus', 1: 'leo varadkar ..."
293,2020-03-13,90,Ireland,"{0: 'coronavirus memes', 1: 'coronavirus death..."


In [14]:
related_queries.related_queries[82]

"{0: 'nsw lockdown coronavirus', 1: 'access super coronavirus', 2: 'coronavirus supplements', 3: 'what are the symptoms of coronavirus', 4: 'coronavirus cases worldwide'}"

In [11]:
trends_title['0'][0]

"{0: 'Powerball tonight', 1: 'Powerball winner', 2: 'Qld school closures', 3: 'New Zealand news', 4: 'Stonnington', 5: 'Stage 3 lockdown Victoria', 6: 'Bugzy Malone', 7: 'Floyd Cardoz', 8: 'Qld schools', 9: 'Fight club', 10: 'Prince Harry', 11: 'Mark Zahra', 12: 'Smiggle', 13: 'Until tomorrow', 14: 'Hairdressers', 15: 'Hairdresser', 16: 'The post', 17: 'Stimulus payment', 18: 'SIA', 19: 'Iceland'}"

In [17]:
trends_title=trends_title.convert_dtypes()
trends_title.dtypes

date            datetime64[ns]
country_code            string
0                       string
dtype: object

In [ ]:
# filtering the ecdc dataframe with the restricted countries for our project

countries = interest.Country.unique()

final_df = ecdc.loc[ecdc['countriesAndTerritories'].isin(countries)]

print("final_df shape:",final_df.shape)
final_df.head()

In [ ]:
# cleaning final_df columns

final_df.rename(columns={'dateRep':'date','countriesAndTerritories':'country','geoId':'country_code'},inplace=True)


In [ ]:
# cleaning the interest dataframe to make it matched with final when merging 

interest.rename(columns={'Country':'country'},inplace=True)

In [ ]:
# cleaning the related_queries dataframe for merging

related_queries.rename(columns={'Country':'country'},inplace=True)

In [ ]:
# cleaning of dataframes for merging

trends_traffic.rename(columns={'0':'trends_traffic'},inplace=True)
trends_title.rename(columns={'0':'trends_title'},inplace=True)

trends_title.head()

In [ ]:
# Merging interest dataframe

result = pd.merge(final_df, interest, how='left', on=['date','country'])
result.head()

In [ ]:
# Merging related_queries dataframe

result_2 = pd.merge(result, related_queries, how='left', on=['date','country','coronavirus'])
result_2.head()

In [ ]:
# Merging trends_title dataframe

result_3 = pd.merge(result_2, trends_title, how='left', on=['date','country_code'])
result_3.head()

In [ ]:
# Merging trends_traffic dataframe

result_4 = pd.merge(result_3, trends_traffic, how='left', on=['date','country_code'])
result_4.head()

In [ ]:
# Creating a new column for the cumulative numbers of cases per country

result_4['cumsum_case'] = result_4.sort_values('date').groupby(['country'])['cases'].transform(lambda g: g.cumsum())
result_4.head()

In [ ]:
# Creating a new column for the daily growth rate of cases per country

result_4['growth_rate']=(result_4['cases']*100)/result_4['cumsum_case']
result_4.growth_rate.fillna(0, inplace=True)

In [ ]:
result_4.isna().sum()

In [ ]:
# saving the final result as csv 

result_4.to_csv('data/final_dataframe.csv', index=False)